# start

In [1]:
SEED_START = 100
n_seeds = 40

PRFX = 'Rndsd0816'

p_o = f'../output/{PRFX}'

# p_o = f'.'

from pathlib import Path
Path(p_o).mkdir(exist_ok=True, parents=True)


!ls $p_o

models


In [2]:
dbg = False
if dbg: dbgsz=1000

from fastai.vision import * 

In [3]:
!pip install ../input/efficientnetpytorch/efficientnet_pytorch-0.3.0-py3-none-any.whl

In [4]:
!nvidia-smi

Fri Aug 16 18:12:43 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   46C    P0    42W / 300W |     10MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# params

In [5]:
BS = 16
FP16 = True
PERC_VAL = 0.1
WD = 0.01


MODEL_NAME = 'efficientnet-b5'
from efficientnet_pytorch import EfficientNet
SZ = EfficientNet.get_image_size(MODEL_NAME)
for i in range(6):
    print(f'efficientnet-b{i} size', EfficientNet.get_image_size(f'efficientnet-b{i}'))
print('SZ:', SZ)

efficientnet-b0 size 224
efficientnet-b1 size 240
efficientnet-b2 size 260
efficientnet-b3 size 300
efficientnet-b4 size 380
efficientnet-b5 size 456
SZ: 456


## img proc

In [6]:
use_open_yz = True

from fastai.vision import *
import cv2
def load_ben_color(fn)->Image:
    image = cv2.imread(fn)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = crop_image_from_gray(image)
    image, _ = crop_margin(image)
    image = center_crop(image)
    image = cv2.resize(image, (640, 480))#most common in test
#     image = cv2.resize(image, (SZ, SZ))
    image = cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX=10) , -4 ,128)
    return image

> get_transforms(do_flip:bool=True, flip_vert:bool=False, max_rotate:float=10.0, max_zoom:float=1.1, max_lighting:float=0.2, max_warp:float=0.2, p_affine:float=0.75, p_lighting:float=0.75, xtra_tfms:Optional[Collection[Transform]]=None) → Collection[Transform]

In [7]:
params_tfms = dict(
    do_flip=True,
    flip_vert=True,
    max_rotate=360,
)

> By default, the library resizes the image while keeping its original ratio so that the smaller size corresponds to the given size, then takes a crop (ResizeMethod.CROP). You can choose to resize the image while keeping its original ratio so that the bigger size corresponds to the given size, then take a pad (ResizeMethod.PAD). Another way is to just squish the image to the given size (ResizeMethod.SQUISH).

In [8]:
kwargs_tfms = dict(
    resize_method=ResizeMethod.SQUISH,
    padding_mode='zeros'
)

# setup

In [9]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.utils.mod_display import *
import datetime

## set seed

In [10]:
def set_torch_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
#         torch.backends.cudnn.deterministic = True
#         torch.backends.cudnn.benchmark = False

## image processing

In [11]:
import cv2

def crop_margin(image, keep_less=0.83):
    
    output = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret,gray = cv2.threshold(gray,10,255,cv2.THRESH_BINARY)
    contours,hierarchy = cv2.findContours(gray,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        #print('no contours!')
        flag = 0
        return image, flag
    cnt = max(contours, key=cv2.contourArea)
    ((x, y), r) = cv2.minEnclosingCircle(cnt)
    r = r*keep_less
    x = int(x); y = int(y); r = int(r)
    flag = 1
    #print(x,y,r)
    if r > 100:
        return output[0 + (y-r)*int(r<y):-1 + (y+r+1)*int(r<y),0 + (x-r)*int(r<x):-1 + (x+r+1)*int(r<x)], flag
    else:
        #print('none!')
        flag = 0
        return image,flag

    
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img
    
# https://stackoverflow.com/questions/16646183/crop-an-image-in-the-centre-using-pil
def center_crop(img):        
    
    h0, w0 = 480, 640 #most common in test
    ratio = h0/w0 #most common in test
    height, width, _= img.shape
    new_width, new_height = width, math.ceil(width*ratio)

    width = img.shape[1]
    height = img.shape[0]

    if new_width is None:
        new_width = min(width, height)

    if new_height is None:
        new_height = min(width, height)

    left = int(np.ceil((width - new_width) / 2))
    right = width - int(np.floor((width - new_width) / 2))

    top = int(np.ceil((height - new_height) / 2))
    bottom = height - int(np.floor((height - new_height) / 2))

    if len(img.shape) == 2:
        center_cropped_img = img[top:bottom, left:right]
    else:
        center_cropped_img = img[top:bottom, left:right, ...]

    return center_cropped_img

def open_yz(fn, convert_mode, after_open)->Image:
    image = load_ben_color(fn)
    return Image(pil2tensor(image, np.float32).div_(255))
    
if use_open_yz:
    vision.data.open_image = open_yz

## QWK

## TTTA

# preprocess

## prep

In [12]:
img2grd = []

p = '../input/aptos2019-blindness-detection'
pp = Path(p)
train = pd.read_csv(pp/'train.csv')
test  = pd.read_csv(pp/'test.csv')
len_blnd = len(train)
len_blnd_test = len(test)

img2grd_blnd = [(f'{p}/train_images/{o[0]}.png',o[1],'blnd')  for o in train.values]

len_blnd, len_blnd_test

(3662, 1928)

In [13]:
img2grd += img2grd_blnd
display(len(img2grd))
cnt = Counter(o[1] for o in img2grd)
t2c_trn_has = dict(cnt)
display(cnt.most_common())
sm = sum(cnt.values())
display([(o[0], o[1]/sm) for o in cnt.most_common()])

3662

[(0, 1805), (2, 999), (1, 370), (4, 295), (3, 193)]

[(0, 0.4929000546149645),
 (2, 0.272801747678864),
 (1, 0.1010376843255052),
 (4, 0.08055707263790278),
 (3, 0.052703440742763515)]

In [14]:
p = '../input/diabetic-retinopathy-detection'
pp = Path(p)
train=pd.read_csv(pp/'trainLabels.csv')
test=pd.read_csv(pp/'retinopathy_solution.csv')

img2grd_diab_train=[(f'{p}/train_images/{o[0]}.jpeg', o[1], 'diab')  for o in train.values]
img2grd_diab_test =[(f'{p}/test_images/{o[0]}.jpeg',  o[1], 'diab')  for o in test.values]
img2grd += img2grd_diab_train
display(len(img2grd))
display(Counter(o[1] for o in img2grd).most_common())
img2grd += img2grd_diab_test
display(len(img2grd))
display(Counter(o[1] for o in img2grd).most_common())

38788

[(0, 27615), (2, 6291), (1, 2813), (3, 1066), (4, 1003)]

92364

[(0, 67148), (2, 14152), (1, 6575), (3, 2280), (4, 2209)]

In [15]:
p = '../input/IDRID/B. Disease Grading'
pp = Path(p)
train=pd.read_csv(pp/'2. Groundtruths/a. IDRiD_Disease Grading_Training Labels.csv')
test=pd.read_csv(pp/'2. Groundtruths/b. IDRiD_Disease Grading_Testing Labels.csv')

img2grd_idrid_train=[(f'{p}/1. Original Images/a. Training Set/{o[0]}.jpg', o[1], 'idri')  for o in train.values]
img2grd_idrid_test =[(f'{p}/1. Original Images/b. Testing Set/{o[0]}.jpg',  o[1], 'idri')  for o in test.values]
img2grd += img2grd_idrid_train
display(len(img2grd))
display(Counter(o[1] for o in img2grd).most_common())
img2grd += img2grd_idrid_test
display(len(img2grd))
display(Counter(o[1] for o in img2grd).most_common())

92777

[(0, 67282), (2, 14288), (1, 6595), (3, 2354), (4, 2258)]

92880

[(0, 67316), (2, 14320), (1, 6600), (3, 2373), (4, 2271)]

In [16]:
df = pd.DataFrame(img2grd)
df.columns = ['fnm', 'target', 'src']
df = df.reset_index()
df.shape

(92880, 4)

In [17]:
df.sample(5).values

array([[54316, '../input/diabetic-retinopathy-detection/test_images/12883_left.jpeg', 0, 'diab'],
       [46474, '../input/diabetic-retinopathy-detection/test_images/6366_left.jpeg', 0, 'diab'],
       [45766, '../input/diabetic-retinopathy-detection/test_images/5786_left.jpeg', 4, 'diab'],
       [37235, '../input/diabetic-retinopathy-detection/train_images/42414_right.jpeg', 0, 'diab'],
       [91935, '../input/diabetic-retinopathy-detection/test_images/43970_right.jpeg', 0, 'diab']], dtype=object)

In [18]:
if not np.all([Path(o[0]).exists() for o in img2grd]): print('Some files are missing!!!')

In [19]:
p = '../input/aptos2019-blindness-detection'
pp = Path(p)
test  = pd.read_csv(pp/'test.csv')

if dbg: test = test.head(dbgsz)

# rndsd

In [20]:
def get_df2use(df, seed):

    df2use = df[df.src=='blnd'].copy()

    def get_randint(low=200, high=1000):
        res = np.random.randn()*300+600
        return int(min(max(low, res), high))

    set_torch_seed(seed)
    n_t_extra = {2:get_randint(),
                 3:get_randint(),
                 4:get_randint(),
                 1:get_randint()
                }
    print('n_t_extra:', n_t_extra)

    set_torch_seed(seed)
    for t,n in n_t_extra.items():
        df_t_non_blnd = df[(df.target==t) & (df.src!='blnd')]
        df2use = pd.concat([df2use, df_t_non_blnd.sample(min(n, len(df_t_non_blnd)))])

    return df2use

In [21]:
tfms = get_transforms(**params_tfms)

In [22]:
def get_data(sz=SZ, bs=BS):
    src = (ImageList.from_df(df=df2use,path='./',cols='fnm') 
            .split_none()
#             .split_from_df(col='is_val')
#             .split_by_rand_pct(0.2)
            .label_from_df(cols='target',  
                          )
          )

    data= (src.transform(tfms, size=sz,
                         **kwargs_tfms
                         ) #Data augmentation
            .databunch(bs=bs) #DataBunch
            .normalize(imagenet_stats) #Normalize     
           )
    return data

## loop

In [ ]:
for seed in range(SEED_START, SEED_START + n_seeds):
    print('seed:', seed, datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    df2use = get_df2use(df, seed)
    if dbg: df2use = df2use.sample(dbgsz)

    set_torch_seed(seed)
    data = get_data()
    data.add_test(ImageList.from_df(test,
                                    '../input/aptos2019-blindness-detection',
                                    folder='test_images',
                                    suffix='.png'))
    model = EfficientNet.from_pretrained(MODEL_NAME, num_classes=5) 
    learn = Learner(data, model, path=p_o)
    if FP16: learn = learn.to_fp16()
    
    set_torch_seed(seed)
    with progress_disabled_ctx(learn) as learn: 
        learn.fit_one_cycle(10, max_lr=1e-3)
    learn.save(f'{PRFX}_mdl_seed_{seed}')
    
    with progress_disabled_ctx(learn) as learn: 
        preds_tst, _ = learn.get_preds(ds_type=DatasetType.Test)
    preds_tst = preds_tst.numpy().squeeze()
    preds_tst_lbls = np.argmax(preds_tst, 1)
    display(pd.Series(preds_tst_lbls).value_counts())
    
    del learn, model
    gc.collect()

seed: 100 2019-08-16 18:12:45
n_t_extra: {2: 200, 3: 702, 4: 945, 1: 524}
Loaded pretrained weights for efficientnet-b5


2    1155
0     344
1     216
4     107
3     106
dtype: int64

seed: 101 2019-08-16 19:26:56
n_t_extra: {2: 1000, 3: 788, 4: 872, 1: 751}
Loaded pretrained weights for efficientnet-b5
